<h1>Imports and Initial Variable Declarations</h1>

In [600]:
# BeautifulSoup and requests library import
from bs4 import BeautifulSoup
import requests 
import re

In [601]:
#page url
page_link = "https://medium.com/@cvonhassett/elizabeth-peyton-aire-and-angels-6072f9891dca"

In [602]:
#page request
page_response = requests.get(page_link, timeout=5)

In [603]:
#parsing content from page and storing it
page_content = BeautifulSoup(page_response.content, "html.parser")

In [604]:
#empty list to store user data
data = []

<h1>Finding and Adding Title to Data</h1>

In [605]:
#grabs div that stores title info
titleInfo = page_content.find_all('div',attrs={"class":"n p"})

In [606]:
#finds title within titleInfo div
href = str(titleInfo).index("<h1")
start = str(titleInfo)[href:].index(">")+href+1
end = str(titleInfo)[href:].index("</h1>")+href

#appends title to data list after removing extra tags
title = re.sub(re.compile('<.*?>'), '', str(titleInfo)[start:end])
data.append(title)

<h1>Finding and Adding Author to Data</h1>

In [607]:
#grabs div that stores user info
userInfo = page_content.find_all('div',attrs={"class":"o n"})

In [608]:
#finds authors name within userInfo div
href = str(userInfo)[24:].index("<a class=")+24
start = str(userInfo)[href:].index(">")+href+1
end = str(userInfo)[href:].index("</a>")+href

#appends authors name to data list
author = str(userInfo)[start:end]
data.append(author)

<h1>Finding and Adding Author Page URL to Data</h1>

In [609]:
#finds date within userInfo div
href = str(userInfo)[24:].index("<a class=")
start = str(userInfo)[href:].index("href=")+href+6
end = str(userInfo)[start:].index("?source")+start

#appends date to data list
url = "medium.com" + str(userInfo)[start:end]
data.append(url)

<h1>Finding and Adding Date to Data</h1>

In [610]:
#finds date within userInfo div
href = str(userInfo)[end:].index("<a class=")+end
start = str(userInfo)[href:].index(">")+href+1
end = str(userInfo)[href:].index("</a>")+href

#appends date to data list
date = str(userInfo)[start:end]
data.append(date)

<h1>Finding and Adding Number of Claps to Data</h1>

In [611]:
#grabs all paragraphs in the article
clapInfo = page_content.find_all('div',attrs={"class":"n o"})

In [612]:
#finds claps within clapInfo div
result = re.sub(re.compile('<.*?>'), '', str(clapInfo))
result = re.findall("\d+", result)

#appends claps to data list
if len(result) == 0:
    data.append('0')

else:
    data.append(result[0])

<h1>Adding a List of the Text Content to Data</h1>

In [613]:
#grabs all paragraphs in the article
textContent = page_content.find_all("p")
textInfo = str(textContent)

#RegEx that cleans paragraph data by removing html tags and extra commas
result = re.sub(re.compile('<.*?>'), '', textInfo)
result = re.split("\., ", result)
result = result[:-1]
data.append(result)
data

['Elizabeth Peyton: Aire and Angels',
 'Riot Material',
 'medium.com/@cvonhassett',
 'Nov 6',
 '0',
 ['[at National Portrait Gallery, London, through 5 January 2020Reviewed by Christopher P Jones, In her new solo show at the National Portrait Gallery in London, New York artist Elizabeth Peyton offers a procession of glistening vignettes, portraits of famous and not-so-famous faces, whose cool freshness leaves the visitor with an excited, if slightly outmoded sense of pursuit. Many of her male subjects carry a melancholy, morning-after expression that signals an unabashed adoption of the female gaze. Peyton has painted Kurt Cobain and David Bowie, David Hockney and scenes from the Twilight films, as well as personal friends and lovers. Her eye tends towards the sheen of beauty, where cheekbones are elevated and noses are streamlined',
  'If the trend in contemporary art is towards the bold and brazen, then Peyton is an outlier, since every pallid-cheeked portrait here has the diminutive